In [1]:
import os
import glob
import matplotlib.pyplot as plt
from tensorflow import keras
import keras_cv

2023-12-14 15:42:45.724784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 15:42:45.724823: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 15:42:45.724847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 15:42:45.730396: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 15:42:46.422164: W tensorflow/compiler/

Using TensorFlow backend


In [2]:
IMAGES_DIR = "/app/data/images/"

dir_names = os.listdir(IMAGES_DIR)
img_paths_dict = dict()
for dir_name in dir_names:
    img_paths_dict[dir_name] = glob.glob(f"{IMAGES_DIR}{dir_name}/*")

FileNotFoundError: [Errno 2] No such file or directory: '/app/data/images/'

In [ ]:
for dir_name, file_paths in img_paths_dict.items():
    print(dir_name, len(file_paths))
    for file_path in file_paths[:5]:
        print(file_path)

In [ ]:
for dir_name, file_paths in img_paths_dict.items():
    print(dir_name)
    for file_path in file_paths[:5]:
        image = plt.imread(file_path)
        plt.imshow(image)
        plt.show()

In [ ]:
dataset = keras.utils.image_dataset_from_directory(IMAGES_DIR)

In [ ]:
images, labels = next(iter(dataset))

print(f"type: {type(images)}")
print(f"dtype: {images.dtype}")
print(f"shape: {images.shape}")

plt.imshow(images[0].numpy().astype("uint8"))
plt.show()

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    IMAGES_DIR, validation_split=0.2, subset="training", label_mode="categorical", seed=0
)
valid_ds = keras.utils.image_dataset_from_directory(
    IMAGES_DIR, validation_split=0.2, subset="validation", label_mode="categorical", seed=0
)

In [ ]:
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset("efficientnetv2_b0_imagenet")

model = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=3,
    activation="softmax",
)

In [ ]:
loss = keras.losses.CategoricalCrossentropy()
metric = keras.metrics.Accuracy()

In [ ]:
model.compile(optimizer="rmsprop", loss=loss, metrics=[metric])

In [ ]:
epochs = 10
model.fit(train_ds, validation_data=valid_ds, epochs=epochs)

In [ ]:
images, labels = next(iter(valid_ds))
preds = model.predict(images)
preds.shape